In [ ]:
# default_exp core

# Data_preprocess

## Objective: 
> The objective of this notebook is to preprocess data to an usable format for Text Mining/NLP/Exploration. In series of steps we would try to do the following steps:
>> - Subset usefull columns of lesson feedback xlsx file
>> - Combine columns of "Wish", "Star", "Feedback"
>> - Clean the text: remove punctuations, stopwords, special characters
>> - Perform Tokenisation
>> - Perform Lemmetisation
>> - Tag Part of Speech
>> - Save the table for further analysis

## Import libraries and load data 

In [ ]:
#export
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)
from nltk import sent_tokenize
import nltk
# import gensim
import string
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm
import spacy
import re
nlp_en = spacy.load("en_core_web_sm")

In [ ]:
path = r'C:\Users\cha95906\OneDrive - Mott MacDonald\SSO Lesson observations'
fn = 'lehwilan-SSO_Classroom_Observation_all_data_original_clean.xlsx'
df = pd.read_excel(os.path.join(path,fn))
print(df.shape)
df.head(3)

(69200, 73)


,gps-coordinates.latitude,gps-coordinates.longitude,gps-coordinates.accuracy,location.region_label,location.district_label,location.ward_label,location.school_label,location.school_emis_code,location.school_new_emis_code,location.school_school_level,location.school_ownership,location.school_town,location.school_chiefdom,date_start,time_start,cd_teacher_name,cd_class,cd_subject,cd_term,cd_lesson,cd_school_gender,cd_girlsenrolled,cd_boysenrolled,cd_girlspresent,cd_boyspresent,lesson_start_date,lesson_start_time,co_star1,co_star1_example,co_star2,co_star2_example,co_wish,co_wish_actions,co_lpm_present,co_lpm_follow,co_lpm_step1,co_lpm_step2,co_lpm_step3,co_lpm_step4,co_lpm_step5,co_positive_language,co_organize,co_mult_assess,co_include,co_content_know,co_feedback,Pedagogy composite score (max of 24),feedback_notes,user-profile.item-1.first_name,user-profile.item-1.last_name,cd_junk_var,co_notes,co_pupil_handbooks,co_fqse_learning_materials,user-profile.item-1.user_role,item_0d659cd377d84f82b34fae0f4215bdc1_firstOpenTime,item_0d659cd377d84f82b34fae0f4215bdc1_disabled,mark_endtime_checkbox,date_end,enumerator,time_end,time_end_unix,user-profile.item-1.tabletUserName,tabletUserName,cd_lwlsupport,cd_lessonvalid,cd_feedbacktype,cd_reponse,cd_previousfeedback,cd_feedbackprogress,co_use_aid,co_use_aid_specify,co_encourage
0,7.959370,-11.743074,4.551,Southern,Bo,277,Kaku A Community Junior Secondary School,319101325.0,NaN,Junior Secondary,Community,Bo,Kakua,2019-10-16,08:39:00,Moses Simbo,jss1,math,1.0,7.0,mixed,12.0,16.0,12.0,10.0,2019-10-16,08:42:00,Teacher involved pupils in to the lesson,Teacher involved pupils in the lesson during t...,Teacher avoid flogging,Teacher did not flog pupils during the lesson.,Teacher did not scanned the class during the o...,SSO engaged and encourage the teacher to pleas...,1.0,2.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,3.0,2.0,2.0,13,Teacher did not scanned the class during the o...,Francess,Foray,NaN,Some of the pupils are not with the pupils han...,some,some,sso,NaN,0.0,1.0,2019-10-16,999.0,09:10:00,999.0,FrancessForay,FrancessForay,Old,Yes,Verbal,Teacher agreed to improve on his wish next les...,First lesson observation,No,999.0,999,NaN
1,NaN,NaN,NaN,Western,Western Area Urban,426,Baptist Junior Secondary School,420102301.0,NaN,Junior Secondary,Mission Religious Group,Bambara Town,Central I,2020-02-12,10:15:00,Mr David Kamara,jss1,english,2.0,55.0,mixed,25.0,23.0,25.0,23.0,2020-02-12,10:17:00,Content,The teacher is comfortable and in control of t...,Assessment,The teacher uses different methods of assessin...,To be more audible in the next lesson,Feedback session with teacher after observation,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,5,NaN,Mohamed Joseph,Kamara,NaN,PHB shortage,most,most,sso,NaN,0.0,1.0,2020-02-12,999.0,10:56:00,999.0,MjJ_kamara,MjJ_kamara,NaN,NaN,NaN,NaN,NaN,NaN,0.0,999,NaN
2,8.825699,-13.131853,6.068,North Western,Port Loko,245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-02-12,11:01:00,Kamara Alie Thim,jss2,english,2.0,56.0,mixed,25.0,32.0,21.0,28.0,2019-02-12,11:06:00,Lesson Delivery,The teacher delivered the lesson perfectly and...,Motivating the pupils,The teacher motivating the pupils during the l...,Needs to prepare ahead for better delivering.,The teacher needs to prepare ahead of any less...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,5,Successful observation.,Santigie Alie,Kamara,NaN,Successful observation.,most,most,sso,NaN,0.0,1.0,2019-02-12,999.0,11:25:00,999.0,SantigieAlieKamara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Sentences at the end of the lesson plan,NaN


## Create a subset of table with relevant columns, the columns selected are as below:
- 'date_start', 'location.district_label', 'location.school_label', 'cd_teacher_name','cd_class','cd_subject','cd_girlspresent',              'cd_boyspresent', 'location.school_school_level', 'co_star1', 'co_star1_example', 'co_star2', 'co_star2_example', 'co_wish',              'co_wish_actions', 'feedback_notes', 'co_lpm_follow', 'Pedagogy composite score (max of 24)'

In [ ]:
df_sub = df[['date_start','location.district_label','location.school_label',
             'cd_teacher_name','cd_class','cd_subject','cd_girlspresent',
             'cd_boyspresent','location.school_school_level','co_star1',
             'co_star1_example','co_star2','co_star2_example','co_wish', 
             'co_wish_actions','feedback_notes','co_lpm_follow','Pedagogy composite score (max of 24)']]

In [ ]:
df_sub.head(3)

,date_start,location.district_label,location.school_label,cd_teacher_name,cd_class,cd_subject,cd_girlspresent,cd_boyspresent,location.school_school_level,co_star1,co_star1_example,co_star2,co_star2_example,co_wish,co_wish_actions,feedback_notes,co_lpm_follow,Pedagogy composite score (max of 24)
0,2019-10-16,Bo,Kaku A Community Junior Secondary School,Moses Simbo,jss1,math,12.0,10.0,Junior Secondary,Teacher involved pupils in to the lesson,Teacher involved pupils in the lesson during t...,Teacher avoid flogging,Teacher did not flog pupils during the lesson.,Teacher did not scanned the class during the o...,SSO engaged and encourage the teacher to pleas...,Teacher did not scanned the class during the o...,2.0,13
1,2020-02-12,Western Area Urban,Baptist Junior Secondary School,Mr David Kamara,jss1,english,25.0,23.0,Junior Secondary,Content,The teacher is comfortable and in control of t...,Assessment,The teacher uses different methods of assessin...,To be more audible in the next lesson,Feedback session with teacher after observation,NaN,4.0,5
2,2019-02-12,Port Loko,NaN,Kamara Alie Thim,jss2,english,21.0,28.0,NaN,Lesson Delivery,The teacher delivered the lesson perfectly and...,Motivating the pupils,The teacher motivating the pupils during the l...,Needs to prepare ahead for better delivering.,The teacher needs to prepare ahead of any less...,Successful observation.,3.0,5


## Combine all star and wish columns

In [ ]:

func = lambda x: x.co_star1 + ". " +x.co_star1_example + ". " + x.co_star2 + ". "+x.co_star2_example+'.'

df_sub['co_star'] = df_sub[['co_star1',
                            'co_star1_example',
                            'co_star2',
                            'co_star2_example']].astype(str).apply(func, axis = 1)

func = lambda x: x.co_wish + ". " +x.co_wish_actions+'.'

df_sub['co_wish'] = df_sub[['co_wish',
                            'co_wish_actions']].astype(str).apply(func, axis = 1)


In [ ]:
df_sub [['co_star','co_wish']].head()

,co_star,co_wish
0,Teacher involved pupils in to the lesson. Teacher involved pupils in the lesson during the guided practice.\nAsk pupils to identify the common multiples of 4&6 .. Teacher avoid flogging. Teacher did not flog pupils during the lesson..,Teacher did not scanned the class during the opening of the lesson.. SSO engaged and encourage the teacher to please scan the class during the lesson..
1,Content. The teacher is comfortable and in control of the content being taught. Assessment. The teacher uses different methods of assessing pupils.,To be more audible in the next lesson. Feedback session with teacher after observation.
2,Lesson Delivery. The teacher delivered the lesson perfectly and explains thoroughly.. Motivating the pupils. The teacher motivating the pupils during the lesson and encourage people to participate fully ..,Needs to prepare ahead for better delivering.. The teacher needs to prepare ahead of any lesson for better understanding..
3,Teacher engaged all pupils in their learning. Teacher treated boys and girls equitably when asking questions.. Teacher structured the lesson properly to allow for pupils active learning.. Teacher used at least one strategy to ensure pupils receive feedback on the individual practice such as peer marking ..,Teacher does proper closing.. Adviced teacher to always recap the entire lesson before leaving the class..
4,The teacher is mobile.. He moves between rows during independent practice.. Teacher speaks audibly and clearly.. He is so audible that pupils at the back can clearly hear him..,The teacher is not comfortable with the Lesson being taught.. I will encourage the teacher to be preparing fully in advance before coming to class..


## Clean the text: remove special characters, punctuation, stop words etc

In [ ]:
def rep(x):
    return re.sub('\s+',' ',x)

fn_remove_punc = lambda x: x.lower().replace('..','.').replace('/',' ').replace('"','')
# fn_rep = lambda x: rep(x)
# .replace(',','.').replace(';','.').replace(' he ','teacher ').replace(' she ','teacher ')

for col in ['co_star','co_wish','feedback_notes']:
    df_sub[col] = df_sub[col].astype(str).apply(fn_remove_punc)
    df_sub[col] = df_sub[col].astype(str).apply(lambda x: rep(x))

In [ ]:
# df_sub.iloc[39686]['co_star']

In [ ]:
for i,j in enumerate(df_sub['co_wish'][0:10]):
    print('\n Sentence: {}, \n Wish: {}'.format(i,j))

In [ ]:
for i,j in enumerate(df_sub['co_star'][0:10]):
    print('\n Sentence: {}, \n Star: {}'.format(i,j))
# i for i in df_sub['co_star'][:10])

In [ ]:
for i,j in enumerate(df_sub['feedback_notes'][0:10]):
    print('\n Sentence: {}, \n Feedback: {}'.format(i,j))
# i for i in df_sub['co_star'][:10])

In [ ]:

translator = str.maketrans('', '', string.punctuation)

In [ ]:
#export
def clean_punc(string,translator):
    string = string.translate(translator)
    
    return string

## remove stopwords
nlp = spacy.load("en_core_web_sm")
# nlp.Defaults.stop_words.add("teacher","student"])
# nlp.Defaults.stop_words |= {"teacher","student","students","lesson","class","pupil","pupils"}

def remove_stopword(string,nlp):
    doc = nlp(string)
    
    token_list = []
    for token in doc:
        token_list.append(token.text)
    
    filtered_sentence =[] 
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    return ' '.join(i for i in filtered_sentence)

In [ ]:
for col in ['co_star','co_wish','feedback_notes']:
    tqdm.pandas()
    df_sub[col+'_clean'] = df_sub[col].astype(str).progress_apply(lambda x: 
                                                        remove_stopword(clean_punc(x,translator),nlp))

In [ ]:
df_sub [['co_star','co_star_clean',"co_wish",'co_wish_clean','feedback_notes','feedback_notes_clean']].head(3)

,co_star,co_star_clean,co_wish,co_wish_clean,feedback_notes,feedback_notes_clean
0,teacher involved pupils in to the lesson. teacher involved pupils in the lesson during the guided practice. ask pupils to identify the common multiples of 4&6 . teacher avoid flogging. teacher did not flog pupils during the lesson.,teacher involved pupils lesson teacher involved pupils lesson guided practice ask pupils identify common multiples 46 teacher avoid flogging teacher flog pupils lesson,teacher did not scanned the class during the opening of the lesson. sso engaged and encourage the teacher to please scan the class during the lesson.,teacher scanned class opening lesson sso engaged encourage teacher scan class lesson,teacher did not scanned the class during the opening of the lesson.,teacher scanned class opening lesson
1,content. the teacher is comfortable and in control of the content being taught. assessment. the teacher uses different methods of assessing pupils.,content teacher comfortable control content taught assessment teacher uses different methods assessing pupils,to be more audible in the next lesson. feedback session with teacher after observation.,audible lesson feedback session teacher observation,nan,nan
2,lesson delivery. the teacher delivered the lesson perfectly and explains thoroughly. motivating the pupils. the teacher motivating the pupils during the lesson and encourage people to participate fully .,lesson delivery teacher delivered lesson perfectly explains thoroughly motivating pupils teacher motivating pupils lesson encourage people participate fully,needs to prepare ahead for better delivering. the teacher needs to prepare ahead of any lesson for better understanding.,needs prepare ahead better delivering teacher needs prepare ahead lesson better understanding,successful observation.,successful observation


## Lemmetise the star, wish and feedback column

In [ ]:
#export
def cal_lemma(string):
    doc = nlp(string)
    y = [x.lemma_ for x in doc]
    return ' '.join(i for i in y)

In [ ]:
cal_lemma(df_sub.co_star_clean[0])

for col in ['co_star','co_wish','feedback_notes']:
    col1 = col+'_clean'
    df_sub[col1+'_lemma'] = df_sub[col1].progress_apply(lambda x: cal_lemma(x) )

In [ ]:
df_sub [['co_star','co_star_clean_lemma',"co_wish",'co_wish_clean_lemma','feedback_notes','feedback_notes_clean_lemma']].head(3)

,co_star,co_star_clean_lemma,co_wish,co_wish_clean_lemma,feedback_notes,feedback_notes_clean_lemma
0,teacher involved pupils in to the lesson. teacher involved pupils in the lesson during the guided practice. ask pupils to identify the common multiples of 4&6 . teacher avoid flogging. teacher did not flog pupils during the lesson.,teacher involve pupil lesson teacher involve pupil lesson guide practice ask pupil identify common multiple 46 teacher avoid flog teacher flog pupil lesson,teacher did not scanned the class during the opening of the lesson. sso engaged and encourage the teacher to please scan the class during the lesson.,teacher scan class opening lesson sso engage encourage teacher scan class lesson,teacher did not scanned the class during the opening of the lesson.,teacher scan class opening lesson
1,content. the teacher is comfortable and in control of the content being taught. assessment. the teacher uses different methods of assessing pupils.,content teacher comfortable control content teach assessment teacher use different method assess pupil,to be more audible in the next lesson. feedback session with teacher after observation.,audible lesson feedback session teacher observation,nan,nan
2,lesson delivery. the teacher delivered the lesson perfectly and explains thoroughly. motivating the pupils. the teacher motivating the pupils during the lesson and encourage people to participate fully .,lesson delivery teacher deliver lesson perfectly explain thoroughly motivate pupil teacher motivate pupil lesson encourage people participate fully,needs to prepare ahead for better delivering. the teacher needs to prepare ahead of any lesson for better understanding.,need prepare ahead well deliver teacher need prepare ahead lesson well understanding,successful observation.,successful observation


## Tag part of Speech for Star, Wish and Feedback

In [ ]:
#export
# calculate POS and Lemma

def set_key(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = value
    elif type(dictionary[key]) == list:
        if value not in dictionary[key]:
            dictionary[key].append(value)
    else:
        dictionary[key] = [dictionary[key], value]
        
def cal_lemma_pos(x):
    s={}
    try:
        doc = nlp(x)
        y=[(x.pos_,x.lemma_) for x in doc]

        for i,j in y:
            set_key(s,i,j)
    except:
        pass
    return s

In [ ]:
tqdm.pandas()
for col in ['co_star','co_wish','feedback_notes']:
#     col1 = col+'_clean'
    df_sub[col+'_pos_lemma'] = df_sub[col].progress_apply(lambda x: cal_lemma_pos(x) )
df_sub.to_csv(os.path.join(path,'cleaned5.csv'))    

In [ ]:
df_sub[['co_star_pos_lemma','co_wish_pos_lemma','feedback_notes_pos_lemma']].head(3)

,co_star_pos_lemma,co_wish_pos_lemma,feedback_notes_pos_lemma
0,"{'NOUN': ['teacher', 'pupil', 'lesson', 'practice', 'multiple'], 'VERB': ['involve', 'involve', 'guide', 'ask', 'identify', 'avoid', 'flog'], 'ADP': ['in', 'to', 'during', 'of'], 'DET': ['the', 'the'], 'PUNCT': ['.', '.'], 'PART': ['to', 'not'], 'ADJ': 'common', 'NUM': '4&6', 'AUX': 'do'}","{'NOUN': ['teacher', 'class', 'opening', 'lesson', 'sso'], 'AUX': 'do', 'PART': ['not', 'to'], 'VERB': ['scan', 'engage', 'encourage', 'please'], 'DET': ['the', 'the'], 'ADP': ['during', 'of'], 'PUNCT': ['.', '.'], 'CCONJ': 'and'}","{'NOUN': ['teacher', 'class', 'opening', 'lesson'], 'AUX': 'do', 'PART': 'not', 'VERB': 'scan', 'DET': ['the', 'the'], 'ADP': ['during', 'of'], 'PUNCT': '.'}"
1,"{'NOUN': ['content', 'teacher', 'control', 'assessment', 'method', 'pupil'], 'PUNCT': ['.', '.'], 'DET': ['the', 'the'], 'AUX': ['be', 'be'], 'ADJ': ['comfortable', 'different'], 'CCONJ': 'and', 'ADP': ['in', 'of'], 'VERB': ['teach', 'use', 'assess']}","{'PART': 'to', 'VERB': 'be', 'ADV': 'more', 'ADJ': ['audible', 'next'], 'ADP': ['in', 'with', 'after'], 'DET': 'the', 'NOUN': ['lesson', 'feedback', 'session', 'teacher', 'observation'], 'PUNCT': ['.', '.']}",{'NOUN': 'nan'}
2,"{'NOUN': ['lesson', 'delivery', 'teacher', 'pupil', 'people'], 'PUNCT': ['.', '.'], 'DET': ['the', 'the'], 'VERB': ['deliver', 'explain', 'motivate', 'encourage', 'participate'], 'ADV': ['perfectly', 'thoroughly', 'fully'], 'CCONJ': ['and', 'and'], 'ADP': 'during', 'PART': 'to'}","{'VERB': ['need', 'prepare'], 'PART': ['to', 'to'], 'ADV': ['ahead', 'ahead'], 'ADP': ['for', 'of'], 'ADJ': ['well', 'well'], 'NOUN': ['delivering', 'teacher', 'lesson', 'understanding'], 'PUNCT': ['.', '.'], 'DET': ['the', 'any']}","{'ADJ': 'successful', 'NOUN': 'observation', 'PUNCT': '.'}"
